In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

workout_df = pd.read_csv('workouts.csv')
workout_df

,title,start_time,end_time,description,exercise_title,superset_id,exercise_notes,set_index,set_type,weight_kg,reps,distance_km,duration_seconds,rpe
0,Legs sábado,"5 Apr 2025, 08:11","5 Apr 2025, 09:40",NaN,Deadlift (Barbell),NaN,NaN,0,normal,180.0,5,NaN,NaN,NaN
1,Legs sábado,"5 Apr 2025, 08:11","5 Apr 2025, 09:40",NaN,Deadlift (Barbell),NaN,NaN,1,failure,180.0,6,NaN,NaN,NaN
2,Legs sábado,"5 Apr 2025, 08:11","5 Apr 2025, 09:40",NaN,Extensão Quadril Banco Romano,NaN,NaN,0,normal,50.0,9,NaN,NaN,NaN
3,Legs sábado,"5 Apr 2025, 08:11","5 Apr 2025, 09:40",NaN,Extensão Quadril Banco Romano,NaN,NaN,1,failure,52.5,7,NaN,NaN,NaN
4,Legs sábado,"5 Apr 2025, 08:11","5 Apr 2025, 09:40",NaN,Seated Leg Curl (Machine),NaN,Pinos 4-6-4,0,normal,55.0,8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6341,Back day - segunda-feira,"11 Dec 2023, 16:54","11 Dec 2023, 19:08",NaN,Calf Press (Machine),NaN,NaN,1,normal,180.0,10,NaN,NaN,NaN
6342,Back day - segunda-feira,"11 Dec 2023, 16:54","11 Dec 2023, 19:08",NaN,Calf Press (Machine),NaN,NaN,2,failure,200.0,10,NaN,NaN,NaN
6343,Back day - segunda-feira,"11 Dec 2023, 16:54","11 Dec 2023, 19:08",NaN,Cable Crunch,NaN,NaN,0,normal,70.0,14,NaN,NaN,NaN
6344,Back day - segunda-feira,"11 Dec 2023, 16:54","11 Dec 2023, 19:08",NaN,Cable Crunch,NaN,NaN,1,normal,70.0,11,NaN,NaN,NaN


In [2]:
workout_df.columns

Index(['title', 'start_time', 'end_time', 'description', 'exercise_title',
       'superset_id', 'exercise_notes', 'set_index', 'set_type', 'weight_kg',
       'reps', 'distance_km', 'duration_seconds', 'rpe'],
      dtype='object')

In [3]:
workout_df.dtypes

title                object
start_time           object
end_time             object
description          object
exercise_title       object
superset_id         float64
exercise_notes       object
set_index             int64
set_type             object
weight_kg           float64
reps                  int64
distance_km         float64
duration_seconds    float64
rpe                 float64
dtype: object

In [4]:
workout_df.describe()

,superset_id,set_index,weight_kg,reps,distance_km,duration_seconds,rpe
count,48.0,6346.000000,6334.000000,6346.000000,0.0,0.0,0.0
mean,0.0,1.172707,70.169452,6.810274,NaN,NaN,NaN
std,0.0,1.103238,54.627999,2.532424,NaN,NaN,NaN
min,0.0,0.000000,1.000000,1.000000,NaN,NaN,NaN
25%,0.0,0.000000,30.000000,5.000000,NaN,NaN,NaN
50%,0.0,1.000000,52.000000,7.000000,NaN,NaN,NaN
75%,0.0,2.000000,100.000000,8.000000,NaN,NaN,NaN
max,0.0,6.000000,330.000000,69.000000,NaN,NaN,NaN


In [5]:
workout_df_columns_filtered = workout_df[['title','start_time', 'exercise_title', 'set_index', 'set_type', 'weight_kg', 'reps']]
workout_df_columns_filtered = workout_df_columns_filtered.rename(columns={'start_time': 'date'})
workout_df_columns_filtered

,title,date,exercise_title,set_index,set_type,weight_kg,reps
0,Legs sábado,"5 Apr 2025, 08:11",Deadlift (Barbell),0,normal,180.0,5
1,Legs sábado,"5 Apr 2025, 08:11",Deadlift (Barbell),1,failure,180.0,6
2,Legs sábado,"5 Apr 2025, 08:11",Extensão Quadril Banco Romano,0,normal,50.0,9
3,Legs sábado,"5 Apr 2025, 08:11",Extensão Quadril Banco Romano,1,failure,52.5,7
4,Legs sábado,"5 Apr 2025, 08:11",Seated Leg Curl (Machine),0,normal,55.0,8
...,...,...,...,...,...,...,...
6341,Back day - segunda-feira,"11 Dec 2023, 16:54",Calf Press (Machine),1,normal,180.0,10
6342,Back day - segunda-feira,"11 Dec 2023, 16:54",Calf Press (Machine),2,failure,200.0,10
6343,Back day - segunda-feira,"11 Dec 2023, 16:54",Cable Crunch,0,normal,70.0,14
6344,Back day - segunda-feira,"11 Dec 2023, 16:54",Cable Crunch,1,normal,70.0,11


In [6]:
workout_df_columns_filtered['date'] = pd.to_datetime(workout_df_columns_filtered['date'], format="%d %b %Y, %H:%M")
# workout_df_columns_filtered['date'] = workout_df_columns_filtered['date'].dt.date
workout_df_columns_filtered['date'] = workout_df_columns_filtered['date'].dt.date
# Data atual
hoje = datetime.now().date()

# Data limite
limite = hoje - timedelta(days=30)

# Filtrando
latest_30_days = workout_df_columns_filtered[workout_df_columns_filtered['date'] >= limite]
latest_30_days

,title,date,exercise_title,set_index,set_type,weight_kg,reps
0,Legs sábado,2025-04-05,Deadlift (Barbell),0,normal,180.0,5
1,Legs sábado,2025-04-05,Deadlift (Barbell),1,failure,180.0,6
2,Legs sábado,2025-04-05,Extensão Quadril Banco Romano,0,normal,50.0,9
3,Legs sábado,2025-04-05,Extensão Quadril Banco Romano,1,failure,52.5,7
4,Legs sábado,2025-04-05,Seated Leg Curl (Machine),0,normal,55.0,8
...,...,...,...,...,...,...,...
225,Legs sábado,2025-03-15,Leg Extension (Machine),1,normal,120.0,8
226,Legs sábado,2025-03-15,Leg Extension (Machine),2,failure,122.5,6
227,Legs sábado,2025-03-15,Seated Calf Raise,0,normal,87.5,11
228,Legs sábado,2025-03-15,Seated Calf Raise,1,normal,87.5,10


In [7]:
latest_30_days[(latest_30_days['title'] == 'Legs sábado') & (latest_30_days['exercise_title'] == 'Deadlift (Barbell)')]

,title,date,exercise_title,set_index,set_type,weight_kg,reps
0,Legs sábado,2025-04-05,Deadlift (Barbell),0,normal,180.0,5
1,Legs sábado,2025-04-05,Deadlift (Barbell),1,failure,180.0,6
71,Legs sábado,2025-03-29,Deadlift (Barbell),0,normal,200.0,3
72,Legs sábado,2025-03-29,Deadlift (Barbell),1,failure,180.0,5
142,Legs sábado,2025-03-22,Deadlift (Barbell),0,normal,200.0,4
143,Legs sábado,2025-03-22,Deadlift (Barbell),1,failure,200.0,3
217,Legs sábado,2025-03-15,Deadlift (Barbell),0,normal,200.0,5
218,Legs sábado,2025-03-15,Deadlift (Barbell),1,failure,200.0,3


In [8]:
fig = go.Figure()

# Add série 0
serie_0 = deadlift_df[deadlift_df['set_index'] == 0]
fig.add_trace(go.Scatter(
    x=serie_0['date'],
    y=serie_0['weight_kg'],
    mode='lines+markers+text',
    name='Série 1',
    text=serie_0['reps'],
    textposition='top center'
))

# Add série 1
serie_1 = deadlift_df[deadlift_df['set_index'] == 1]
fig.add_trace(go.Scatter(
    x=serie_1['date'],
    y=serie_1['weight_kg'],
    mode='lines+markers+text',
    name='Série 2',
    text=serie_1['reps'],
    textposition='top center',
    visible=False
))

# Dropdown para controlar visibilidade
fig.update_layout(
    title='Evolução no Deadlift (Barbell)',
    xaxis_title='Data',
    yaxis_title='Peso (kg)',
    updatemenus=[
        {
            "buttons": [
                {"label": "Série 1", "method": "update", "args": [{"visible": [True, False]}]},
                {"label": "Série 2 - Top set", "method": "update", "args": [{"visible": [False, True]}]},
                #{"label": "Ambas", "method": "update", "args": [{"visible": [True, True]}]},
            ],
            "direction": "down",
        }
    ]
)

fig.show()

NameError: name 'deadlift_df' is not defined

In [9]:
import plotly.graph_objects as go
import pandas as pd

def gerar_grafico_interativo(df, treino_nome, exercicio_nome, output_path='grafico_treino.html'):
    # Cria uma cópia do DataFrame para evitar modificações no original
    df = df.copy()
    
    # Garante que a coluna 'date' está no formato datetime.date
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Filtra pelo exercício e treino desejado
    exercicio_df = df[(df['title'] == treino_nome) & (df['exercise_title'] == exercicio_nome)]

    # Verifica se há dados após o filtro
    if exercicio_df.empty:
        print(f"Nenhum dado encontrado para {exercicio_nome} em {treino_nome}")
        return None

    # Calcula a data máxima para referência
    max_date = exercicio_df['date'].max()

    # Filtra inicialmente para os últimos 30 dias (padrão)
    initial_data = exercicio_df[exercicio_df['date'] >= (max_date - pd.Timedelta(days=30))]

    # Cria os traces iniciais para os últimos 30 dias
    traces = []
    for set_idx in sorted(initial_data['set_index'].unique()):  # Ordena para consistência
        set_data = initial_data[initial_data['set_index'] == set_idx]
        traces.append(go.Scatter(
            x=set_data['date'],
            y=set_data['weight_kg'],
            mode='lines+markers+text',
            name=f'Série {set_idx + 1}',
            text=[f'{r} reps' for r in set_data['reps']],
            textposition='top center',
            hovertemplate='Data: %{x}<br>Peso: %{y} kg<br>Reps: %{text}<extra></extra>'
        ))

    # Layout com botões de filtro
    layout = go.Layout(
        title=f'Evolução no {exercicio_nome} ({treino_nome})',
        xaxis=dict(title='Data', tickformat='%d/%m/%Y'),
        yaxis=dict(title='Peso (kg)'),
        hovermode='closest',
        updatemenus=[
            {
                "buttons": [
                    {
                        "label": "Últimos 30 dias",
                        "method": "restyle",
                        "args": [{
                            "x": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=30)))]['date'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "y": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=30)))]['weight_kg'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "text": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=30)))]['reps'].apply(lambda r: f'{r} reps') for idx in sorted(exercicio_df['set_index'].unique())],
                            "visible": [True] * len(exercicio_df['set_index'].unique())
                        }],
                        "args2": [{"visible": [True] * len(exercicio_df['set_index'].unique())}],
                    },
                    {
                        "label": "Últimos 60 dias",
                        "method": "restyle",
                        "args": [{
                            "x": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=60)))]['date'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "y": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=60)))]['weight_kg'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "text": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=60)))]['reps'].apply(lambda r: f'{r} reps') for idx in sorted(exercicio_df['set_index'].unique())],
                            "visible": [True] * len(exercicio_df['set_index'].unique())
                        }],
                        "args2": [{"visible": [True] * len(exercicio_df['set_index'].unique())}],
                    },
                    {
                        "label": "Últimos 90 dias",
                        "method": "restyle",
                        "args": [{
                            "x": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=90)))]['date'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "y": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=90)))]['weight_kg'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "text": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=90)))]['reps'].apply(lambda r: f'{r} reps') for idx in sorted(exercicio_df['set_index'].unique())],
                            "visible": [True] * len(exercicio_df['set_index'].unique())
                        }],
                        "args2": [{"visible": [True] * len(exercicio_df['set_index'].unique())}],
                    },
                    {
                        "label": "Todos os dados",
                        "method": "restyle",
                        "args": [{
                            "x": [exercicio_df[exercicio_df['set_index'] == idx]['date'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "y": [exercicio_df[exercicio_df['set_index'] == idx]['weight_kg'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "text": [exercicio_df[exercicio_df['set_index'] == idx]['reps'].apply(lambda r: f'{r} reps') for idx in sorted(exercicio_df['set_index'].unique())],
                            "visible": [True] * len(exercicio_df['set_index'].unique())
                        }],
                        "args2": [{"visible": [True] * len(exercicio_df['set_index'].unique())}],
                    }
                ],
                "direction": "down",
                "showactive": True,
                "active": 0,  # Define "Últimos 30 dias" como o botão ativo por padrão
                "x": 0.1,
                "xanchor": "left",
                "y": 1.2,
                "yanchor": "top"
            }
        ]
    )

    # Cria a figura
    fig = go.Figure(data=traces, layout=layout)
    
    # Salva o gráfico como HTML
    fig.write_html(output_path, auto_open=False)  # auto_open=False para evitar abrir o navegador automaticamente
    
    # Retorna a figura para visualização direta em notebooks
    return fig

# Exemplo de uso:
graf = gerar_grafico_interativo(workout_df_columns_filtered, 'Legs sábado', 'Deadlift (Barbell)')
if graf is not None:
    graf.show()  # Exibe o gráfico diretamente no Jupyter Notebook

In [ ]:
import plotly.graph_objects as go
import pandas as pd
from datetime import timedelta

# Função auxiliar para criar os botões de séries
def create_series_buttons(orders_periodo, all_orders, has_failure):
    buttons = []
    # Top Set
    if has_failure:
        buttons.append({
            "label": "Top Set (Failure)",
            "method": "update",
            "args": [{"visible": [True] + [False]*len(all_orders)}]
        })

    # Work Sets
    for o in orders_periodo:
        buttons.append({
            "label": f"Work Set {int(o)}",
            "method": "update",
            "args": [{"visible": [False] + [True if o == current else False for current in all_orders]}]
        })

    # Todas
    buttons.append({
        "label": "Todas as séries",
        "method": "update",
        "args": [{"visible": [True] + [True if o in orders_periodo else False for o in all_orders]}]
    })

    return buttons

# Função principal com lógica dos botões
def gerar_grafico_interativo_com_series(df, treino_nome, exercicio_nome, output_path='grafico_treino_com_series.html'):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date']).dt.date

    exercicio_df = df[(df['title'] == treino_nome) & (df['exercise_title'] == exercicio_nome)]
    if exercicio_df.empty:
        print("Sem dados.")
        return None

    has_failure = not exercicio_df[exercicio_df['set_type'] == 'failure'].empty
    max_date = exercicio_df['date'].max()

    exercicio_df['normal_order'] = None
    for dt in exercicio_df['date'].unique():
        subset = exercicio_df[(exercicio_df['date'] == dt) & (exercicio_df['set_type'] == 'normal')].sort_values('set_index')
        for i, idx in enumerate(subset.index, 1):
            exercicio_df.loc[idx, 'normal_order'] = i

    exercicio_df['normal_order'] = exercicio_df['normal_order'].astype('Int64')

    all_orders = sorted(exercicio_df[(exercicio_df['normal_order'].notna()) & (exercicio_df['date'] >= (max_date - timedelta(days=90)))]['normal_order'].unique())

    # Criação de traces
    traces = []

    # Top set (visível inicialmente)
    if has_failure:
        fdata = exercicio_df[(exercicio_df['set_type'] == 'failure') & (exercicio_df['date'] >= max_date - timedelta(days=30))]
        traces.append(go.Scatter(
            x=fdata['date'], y=fdata['weight_kg'], text=[f"{r} reps" for r in fdata['reps']],
            mode='lines+markers+text', textposition='top center',
            name='Top Set (Failure)', visible=True
        ))

    # Work sets (invisíveis inicialmente)
    for o in all_orders:
        wdata = exercicio_df[(exercicio_df['normal_order'] == o) & (exercicio_df['date'] >= max_date - timedelta(days=30))]
        traces.append(go.Scatter(
            x=wdata['date'], y=wdata['weight_kg'], text=[f"{r} reps" for r in wdata['reps']],
            mode='lines+markers+text', textposition='top center',
            name=f'Work Set {int(o)}', visible=False
        ))

    # Armazena botões e dados por período
    periodos = [30, 60, 90]
    period_buttons = []

    for dias in periodos:
        periodo_inicio = max_date - timedelta(days=dias)
        orders = sorted(exercicio_df[(exercicio_df['normal_order'].notna()) & (exercicio_df['date'] >= periodo_inicio)]['normal_order'].unique())

        x_list, y_list, text_list = [], [], []

        if has_failure:
            df_f = exercicio_df[(exercicio_df['set_type'] == 'failure') & (exercicio_df['date'] >= periodo_inicio)]
            x_list.append(df_f['date'].tolist())
            y_list.append(df_f['weight_kg'].tolist())
            text_list.append([f"{r} reps" for r in df_f['reps']])
        else:
            x_list.append([]); y_list.append([]); text_list.append([])

        for o in all_orders:
            df_w = exercicio_df[(exercicio_df['normal_order'] == o) & (exercicio_df['date'] >= periodo_inicio)]
            x_list.append(df_w['date'].tolist())
            y_list.append(df_w['weight_kg'].tolist())
            text_list.append([f"{r} reps" for r in df_w['reps']])

        period_buttons.append(dict(
            label=f"Últimos {dias} dias",
            method="update",
            args=[
                {"x": x_list, "y": y_list, "text": text_list},
                {"updatemenus[1].buttons": create_series_buttons(orders, all_orders, has_failure)}
            ]
        ))
    period_buttons[0]["args"][1]["title"] = f"Evolução no {exercicio_nome} ({treino_nome}) - Últimos 30 dias"

    layout = go.Layout(
    title=f'Evolução no {exercicio_nome} ({treino_nome})',
    xaxis=dict(
        title='Data',
        tickformat='%d/%m/%Y'
    ),
    yaxis=dict(
        title='Peso (kg)'
    ),
    hovermode='closest',
    updatemenus=[
        dict(
            buttons=period_buttons,
            direction="down",
            showactive=True,
            active=0,
            x=0.6,  # Mais próximo da borda esquerda
            xanchor="left",
            y=1.2,   # Bem acima do gráfico
            yanchor="top",
            pad={"r": 10, "t": 10},
            bgcolor="white",
            bordercolor="gray",
            borderwidth=1
        ),
        dict(
            buttons=create_series_buttons(
                sorted(exercicio_df[(exercicio_df['normal_order'].notna()) & (exercicio_df['date'] >= max_date - timedelta(days=30))]['normal_order'].unique()),
                all_orders,
                has_failure
            ),
            direction="down",
            showactive=True,
            active=0,
            x=0.99,  # Lado direito
            xanchor="right",
            y=1.2,   # Mesmo nível do outro menu
            yanchor="top",
            pad={"r": 10, "t": 10},
            bgcolor="white",
            bordercolor="gray",
            borderwidth=1
        )
    ]
)

    fig = go.Figure(data=traces, layout=layout)
    #fig.write_html(output_path, auto_open=False)
    #return fig

    # Salva o gráfico como HTML interativo em arquivo (como já faz)
    fig.write_html(output_path, auto_open=False)

    # Também retorna o HTML como string, para inserção em JS ou <div>
    html_string = fig.to_html(full_html=False, include_plotlyjs='cdn')
    return fig, html_string

# Exemplo de uso
graf, _ = gerar_grafico_interativo_com_series(workout_df_columns_filtered, 'Legs sábado', 'Deadlift (Barbell)')
if graf is not None:
    graf.show()


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\1312879778.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\1312879778.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
# Filtrar os exercícios únicos para o treino "Legs sábado"
exercicios_legs_sabado = workout_df_columns_filtered[workout_df_columns_filtered['title'] == 'Legs sábado']['exercise_title'].unique()
def gerar_grafico_treino_total(treino_exercicios, nome_treino='Legs sábado'):
    # Filtra os dados para o treino específico
    print(f"Exercícios na rotina {treino_exercicios}:")
    for exercicio in treino_exercicios:
        print(f"- {exercicio}")
        graf = gerar_grafico_interativo_com_series(workout_df_columns_filtered, nome_treino, exercicio, output_path='grafico_treino.html')
        #if graf is not None:
            #graf.show()
gerar_grafico_treino_total(exercicios_legs_sabado)

Exercícios na rotina ['Deadlift (Barbell)' 'Extensão Quadril Banco Romano'
 'Seated Leg Curl (Machine)' 'Leg Extension (Machine)' 'Seated Calf Raise'
 'Straight Leg Deadlift' 'Standing Leg Curls' 'Hip Thrust (Barbell)'
 'Hip Adduction (Machine)' 'Calf Extension (Machine)'
 'Hip Abduction (Machine)' 'Single Leg Extensions' 'Deadlift High Pull'
 'Lying Leg Curl (Machine)' 'Crunch (Machine)']:
- Deadlift (Barbell)
- Extensão Quadril Banco Romano
- Seated Leg Curl (Machine)
- Leg Extension (Machine)
- Seated Calf Raise


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

- Straight Leg Deadlift
- Standing Leg Curls
- Hip Thrust (Barbell)
- Hip Adduction (Machine)
- Calf Extension (Machine)


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

- Hip Abduction (Machine)
- Single Leg Extensions
- Deadlift High Pull
- Lying Leg Curl (Machine)
- Crunch (Machine)


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [13]:
treino_nomes = workout_df_columns_filtered['title'].unique()
for nome_treino in treino_nomes:
    print(f"\nTreino: {nome_treino}")
    # Filtrar os exercícios únicos para o treino atual
    exercicios = workout_df_columns_filtered[workout_df_columns_filtered['title'] == nome_treino]['exercise_title'].unique()
    gerar_grafico_treino_total(exercicios, nome_treino=nome_treino)
# Salvar o gráfico como HTML


Treino: Legs sábado
Exercícios na rotina ['Deadlift (Barbell)' 'Extensão Quadril Banco Romano'
 'Seated Leg Curl (Machine)' 'Leg Extension (Machine)' 'Seated Calf Raise'
 'Straight Leg Deadlift' 'Standing Leg Curls' 'Hip Thrust (Barbell)'
 'Hip Adduction (Machine)' 'Calf Extension (Machine)'
 'Hip Abduction (Machine)' 'Single Leg Extensions' 'Deadlift High Pull'
 'Lying Leg Curl (Machine)' 'Crunch (Machine)']:
- Deadlift (Barbell)
- Extensão Quadril Banco Romano
- Seated Leg Curl (Machine)
- Leg Extension (Machine)
- Seated Calf Raise


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

- Straight Leg Deadlift
- Standing Leg Curls
- Hip Thrust (Barbell)
- Hip Adduction (Machine)
- Calf Extension (Machine)


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

- Hip Abduction (Machine)
- Single Leg Extensions
- Deadlift High Pull
- Lying Leg Curl (Machine)
- Crunch (Machine)


...

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

- Cable Crunch

Treino: Abdômen
Exercícios na rotina ['Crunch (Machine)']:
- Crunch (Machine)

Treino: Leg day - quarta
Exercícios na rotina ['Squat (Barbell)' 'Bulgarian Split Squat' 'Leg Extension (Machine)'
 'Straight Leg Deadlift' 'Hip Thrust (Barbell)' 'Calf Press (Machine)']:
- Squat (Barbell)
- Bulgarian Split Squat
- Leg Extension (Machine)
- Straight Leg Deadlift
- Hip Thrust (Barbell)
- Calf Press (Machine)

Treino: Back day - segunda-feira
Exercícios na rotina ['Pull Up (Weighted)' 'Iso-Lateral Row (Machine)'
 'Chest Supported Incline Row (Dumbbell)' 'Rear Delt Reverse Fly (Cable)'
 'Cross Body Hammer Curl' 'Bicep Curl (Barbell)' 'Calf Press (Machine)'
 'Cable Crunch']:
- Pull Up (Weighted)
- Iso-Lateral Row (Machine)
- Chest Supported Incline Row (Dumbbell)
- Rear Delt Reverse Fly (Cable)


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

- Cross Body Hammer Curl
- Bicep Curl (Barbell)
- Calf Press (Machine)
- Cable Crunch


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [14]:
import plotly.graph_objects as go
import pandas as pd
from plotly.io import to_html
import os

# Filtrar os exercícios únicos para o treino "Legs sábado"
exercicios_legs_sabado = workout_df_columns_filtered[workout_df_columns_filtered['title'] == 'Legs sábado']['exercise_title'].unique()

print("Exercícios na rotina 'Legs sábado':")
print("- " + "\n- ".join(exercicios_legs_sabado))

# Criar uma lista para armazenar o HTML de cada gráfico
graficos_html = []

# Gerar cada gráfico e capturar seu HTML
for exercicio in exercicios_legs_sabado:
    print(f"- Gerando gráfico para: {exercicio}")
    # Gerar o gráfico sem salvar em disco (output_path=None)
    fig = gerar_grafico_interativo_com_series(workout_df_columns_filtered, 'Legs sábado', exercicio, output_path='grafico_treino.html')
    if fig is not None:
        # Converter o gráfico para HTML
        grafico_html = to_html(fig, full_html=False, include_plotlyjs='cdn')
        graficos_html.append((exercicio, grafico_html))

# Criar o template HTML com abas (mesmo template anterior)
html_template = """
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <title>Todos os Gráficos da Rotina Legs Sábado</title>
    <style>
        .tab {
            overflow: hidden;
            border: 1px solid #ccc;
            background-color: #f1f1f1;
        }

        .tab button {
            background-color: inherit;
            float: left;
            border: none;
            outline: none;
            cursor: pointer;
            padding: 14px 16px;
            transition: 0.3s;
            font-size: 16px;
        }

        .tab button:hover {
            background-color: #ddd;
        }

        .tab button.active {
            background-color: #ccc;
        }

        .tabcontent {
            display: none;
            padding: 6px 12px;
            border: 1px solid #ccc;
            border-top: none;
        }
    </style>
</head>
<body>

<h2>Gráficos de Evolução - Rotina Legs Sábado</h2>

<div class="tab">
"""

# Adicionar botões de aba
for exercicio, _ in graficos_html:
    tab_id = exercicio.replace(" ", "_").replace("(", "").replace(")", "").lower()
    html_template += f'    <button onclick="openTab(event, \'{tab_id}\')">{exercicio}</button>\n'

html_template += """
</div>
"""

# Adicionar conteúdo de cada aba
for exercicio, grafico_html in graficos_html:
    tab_id = exercicio.replace(" ", "_").replace("(", "").replace(")", "").lower()
    html_template += f"""
<div id="{tab_id}" class="tabcontent">
    <h3>{exercicio}</h3>
    {grafico_html}
</div>
"""

html_template += """
<script>
function openTab(evt, tabName) {
    var i, tabcontent, tablinks;
    tabcontent = document.getElementsByClassName("tabcontent");
    for (i = 0; i < tabcontent.length; i++) {
        tabcontent[i].style.display = "none";
    }
    tablinks = document.getElementsByClassName("tab");
    for (i = 0; i < tablinks.length; i++) {
        tablinks[i].className = tablinks[i].className.replace(" active", "");
    }
    document.getElementById(tabName).style.display = "block";
    evt.currentTarget.className += " active";
}

// Mostrar a primeira aba por padrão
document.addEventListener("DOMContentLoaded", function() {
    document.getElementsByClassName("tab")[0].click();
});
</script>

</body>
</html>
"""

# Salvar o HTML final
output_path = 'todos_graficos_legs_sabado_com_abas.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(html_template)

print(f"Todos os gráficos foram salvos em um único arquivo: {output_path}")
print("Abra o arquivo em um navegador para visualizar as abas.")

Exercícios na rotina 'Legs sábado':
- Deadlift (Barbell)
- Extensão Quadril Banco Romano
- Seated Leg Curl (Machine)
- Leg Extension (Machine)
- Seated Calf Raise
- Straight Leg Deadlift
- Standing Leg Curls
- Hip Thrust (Barbell)
- Hip Adduction (Machine)
- Calf Extension (Machine)
- Hip Abduction (Machine)
- Single Leg Extensions
- Deadlift High Pull
- Lying Leg Curl (Machine)
- Crunch (Machine)
- Gerando gráfico para: Deadlift (Barbell)
- Gerando gráfico para: Extensão Quadril Banco Romano
- Gerando gráfico para: Seated Leg Curl (Machine)
- Gerando gráfico para: Leg Extension (Machine)
- Gerando gráfico para: Seated Calf Raise


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

- Gerando gráfico para: Straight Leg Deadlift
- Gerando gráfico para: Standing Leg Curls
- Gerando gráfico para: Hip Thrust (Barbell)
- Gerando gráfico para: Hip Adduction (Machine)
- Gerando gráfico para: Calf Extension (Machine)


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

- Gerando gráfico para: Hip Abduction (Machine)
- Gerando gráfico para: Single Leg Extensions
- Gerando gráfico para: Deadlift High Pull
- Gerando gráfico para: Lying Leg Curl (Machine)
- Gerando gráfico para: Crunch (Machine)
Todos os gráficos foram salvos em um único arquivo: todos_graficos_legs_sabado_com_abas.html
Abra o arquivo em um navegador para visualizar as abas.


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\4064115024.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [32]:
graficos_por_exercicio = {}
for rotina in workout_df_columns_filtered['title'].unique():
    exercicios = workout_df_columns_filtered[workout_df_columns_filtered['title'] == rotina]['exercise_title'].unique()
    for exercicio in exercicios:
        fig, html = gerar_grafico_interativo_com_series(workout_df_columns_filtered, rotina, exercicio)
        if html:
            graficos_por_exercicio[(rotina, exercicio)] = html.replace("`", "\\`")  # escapando crase

graficos_por_exercicio


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\3709795495.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\3709795495.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\3709795495.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

{('Legs sábado',
  'Deadlift (Barbell)'): '<div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n        <script charset="utf-8" src="https://cdn.plot.ly/plotly-3.0.1.min.js"></script>                <div id="21c0473e-a2a2-49ff-a0a8-c02d2a8bf544" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                window.PLOTLYENV=window.PLOTLYENV || {};                                if (document.getElementById("21c0473e-a2a2-49ff-a0a8-c02d2a8bf544")) {                    Plotly.newPlot(                        "21c0473e-a2a2-49ff-a0a8-c02d2a8bf544",                        [{"mode":"lines+markers+text","name":"Top Set (Failure)","text":["6 reps","5 reps","3 reps","3 reps","3 reps"],"textposition":"top center","visible":true,"x":["2025-04-05","2025-03-29","2025-03-22","2025-03-15","2025-03-08"],"y":{"dtype":"f8","bdata":"AAAAAACAZkAAAAAAAIBmQAAAAAAAAGlAAAAAAAAAaUAAAAAA

In [33]:
import json
js_graficos_dict = "const graficosPlotly = {\n"
for (rotina, exercicio), html in graficos_por_exercicio.items():
    js_graficos_dict += f'  "{rotina}||{exercicio}": `{html}`,\n'
js_graficos_dict += "};"


In [35]:
import json

graficos_plotly_data = {}
for (rotina, exercicio), fig in figs_dict.items():
    graficos_plotly_data[f"{rotina}||{exercicio}"] = fig.to_dict()
js_plotly_data = f"const graficosPlotly = {json.dumps(graficos_plotly_data)};"


NameError: name 'figs_dict' is not defined

In [34]:
import json

# Assuming workout_df_columns_filtered is already loaded
# If not, load it here, e.g., workout_df_columns_filtered = pd.read_csv('your_file.csv')

# Step 1: Extract unique routines and their exercises
treino_nomes = workout_df_columns_filtered['title'].unique()
rotinas_exercicios = {}
for nome_treino in treino_nomes:
    exercicios = workout_df_columns_filtered[workout_df_columns_filtered['title'] == nome_treino]['exercise_title'].unique().tolist()
    rotinas_exercicios[nome_treino] = exercicios



# Step 2: Generate JavaScript object as a string
js_rotinas_exercicios = f"const rotinasExercicios = {json.dumps(rotinas_exercicios)};"

# Step 3: Create HTML template
html_template = f"""
<!DOCTYPE html>
<html lang="pt">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Visualização de Treinos</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
        }}
        select {{
            margin: 10px 0;
            padding: 5px;
            font-size: 16px;
        }}
        #grafico-container {{
            width: 100%;
            height: 600px;
            margin-top: 20px;
        }}
    </style>
</head>
<body>
    <h1>Visualização de Progresso nos Treinos</h1>
    <label for="rotina">Rotina de Treino:</label>
    <select id="rotina" onchange="atualizarExercicios()">
        <option value="">Selecione uma rotina</option>
    </select>
    <br>
    <label for="exercicio">Exercício:</label>
    <select id="exercicio" onchange="carregarGrafico()">
        <option value="">Selecione um exercício</option>
    </select>
    <div id="grafico-container">
        <!-- O gráfico será carregado aqui -->
    </div>

    <script>
        {js_rotinas_exercicios}
        {js_graficos_dict}
        // Preencher o dropdown de rotinas
        const rotinaSelect = document.getElementById('rotina');
        for (const rotina in rotinasExercicios) {{
            const option = document.createElement('option');
            option.value = rotina;
            option.text = rotina;
            rotinaSelect.appendChild(option);
        }}

        // Função para atualizar o dropdown de exercícios
        function atualizarExercicios() {{
            const rotina = rotinaSelect.value;
            const exercicioSelect = document.getElementById('exercicio');
            exercicioSelect.innerHTML = '<option value="">Selecione um exercício</option>'; // Resetar opções
            if (rotina && rotinasExercicios[rotina]) {{
                const exercicios = rotinasExercicios[rotina];
                exercicios.forEach(exercicio => {{
                    const option = document.createElement('option');
                    option.value = exercicio;
                    option.text = exercicio;
                    exercicioSelect.appendChild(option);
                }});
                carregarGrafico(); // Carregar gráfico do primeiro exercício, se houver
            }} else {{
                document.getElementById('grafico-container').innerHTML = ''; // Limpar gráfico se nenhuma rotina
            }}
        }}

        // Função para carregar o gráfico
        function carregarGrafico() {{
    const rotina = rotinaSelect.value;
    const exercicio = document.getElementById('exercicio').value;
    const container = document.getElementById('grafico-container');

    if (rotina && exercicio) {{
        const chave = `${rotina}||${exercicio}`;
        if (graficosPlotly[chave]) {{
            container.innerHTML = `<h2>${rotina} - ${exercicio}</h2>` + graficosPlotly[chave];
        }} else {{
            container.innerHTML = `<p>Nenhum gráfico disponível para essa combinação.</p>`;
        }}
    }} else {{
        container.innerHTML = '';
    }}
        }}


        // Inicializar a página
        atualizarExercicios();
    </script>
</body>
</html>
"""

# Step 4: Write HTML to file
with open('index.html', 'w', encoding='utf-8') as f:
    f.write(html_template)

print("Arquivo 'index.html' gerado com sucesso.")

Arquivo 'index.html' gerado com sucesso.


In [53]:
import json

def gerar_graficos_dict(df, rotinas_exercicios):
    graficos_dict = {}

    for rotina, exercicios in rotinas_exercicios.items():
        for exercicio in exercicios:
            fig, _ = gerar_grafico_interativo_com_series(df, rotina, exercicio)
            if fig:
                key = f"{rotina}||{exercicio}"
                graficos_dict[key] = fig.to_dict()
    
    return graficos_dict


In [54]:
graficos_plotly_dict = gerar_graficos_dict(workout_df_columns_filtered, rotinas_exercicios)
from plotly.utils import PlotlyJSONEncoder

js_plotly_data = f"const graficosPlotly = {json.dumps(graficos_plotly_dict, cls=PlotlyJSONEncoder)};"


C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\1312879778.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\1312879778.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_34360\1312879778.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [55]:
html_template = f"""
<!DOCTYPE html>
<html lang="pt">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Visualização de Treinos</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
        }}
        select {{
            margin: 10px 0;
            padding: 5px;
            font-size: 16px;
        }}
        #grafico-container {{
            width: 100%;
            height: 600px;
            margin-top: 20px;
        }}
    </style>
</head>
<body>
    <h1>Visualização de Progresso nos Treinos</h1>
    <label for="rotina">Rotina de Treino:</label>
    <select id="rotina" onchange="atualizarExercicios()">
        <option value="">Selecione uma rotina</option>
    </select>
    <br>
    <label for="exercicio">Exercício:</label>
    <select id="exercicio" onchange="carregarGrafico()">
        <option value="">Selecione um exercício</option>
    </select>
    <div id="grafico-container"></div>

    <script>
        // Dados JS dos gráficos
        {js_plotly_data}

        // Dicionário de rotinas e exercícios
        const rotinasExercicios = {json.dumps(rotinas_exercicios)};

        // Popular o dropdown de rotinas
        const rotinaSelect = document.getElementById('rotina');
        for (const rotina in rotinasExercicios) {{
            const option = document.createElement('option');
            option.value = rotina;
            option.text = rotina;
            rotinaSelect.appendChild(option);
        }}

        // Atualizar exercícios com base na rotina
        function atualizarExercicios() {{
            const rotina = rotinaSelect.value;
            const exercicioSelect = document.getElementById('exercicio');
            exercicioSelect.innerHTML = '<option value="">Selecione um exercício</option>';
            if (rotinasExercicios[rotina]) {{
                rotinasExercicios[rotina].forEach(exercicio => {{
                    const option = document.createElement('option');
                    option.value = exercicio;
                    option.text = exercicio;
                    exercicioSelect.appendChild(option);
                }});
            }}
            carregarGrafico(); // Carrega gráfico automaticamente do primeiro
        }}

        // Carregar gráfico via Plotly.newPlot
        function carregarGrafico() {{
            const rotina = rotinaSelect.value;
            const exercicio = document.getElementById('exercicio').value;
            const chave = rotina + "||" + exercicio;
            const container = document.getElementById('grafico-container');
            container.innerHTML = ""; // Limpa conteúdo antigo
            if (graficosPlotly[chave]) {{
                const fig = graficosPlotly[chave];
                Plotly.newPlot(container, fig.data, fig.layout);
            }}
        }}

        // Inicializar
        atualizarExercicios();
    </script>
</body>
</html>
"""

with open("visualizacao_treino_interativa.html", "w", encoding="utf-8") as f:
    f.write(html_template)
